In [1]:
from config import API_Key
import pandas as pd
import requests
import json
from citipy import citipy

In [5]:
url = f"http://api.airvisual.com/v2/city?city=Los Angeles&state=California&country=USA&key={API_Key}"
response=requests.get(url)
data=response.json()
print(json.dumps(data,indent=4,sort_keys=4))

{
    "data": {
        "city": "Los Angeles",
        "country": "USA",
        "current": {
            "pollution": {
                "aqicn": 33,
                "aqius": 73,
                "maincn": "p2",
                "mainus": "p2",
                "ts": "2022-06-16T08:00:00.000Z"
            },
            "weather": {
                "hu": 85,
                "ic": "50n",
                "pr": 1009,
                "tp": 18,
                "ts": "2022-06-16T08:00:00.000Z",
                "wd": 130,
                "ws": 3.09
            }
        },
        "location": {
            "coordinates": [
                -118.2417,
                34.0669
            ],
            "type": "Point"
        },
        "state": "California"
    },
    "status": "success"
}


In [ ]:
city=citipy.nearest_city()

In [11]:
filepath="Climate_Data/ClimateClassifications.csv"
climate_df=pd.read_csv(filepath)
climate_df

,Lat,Lon,Cls
0,-89.75,-179.75,EF
1,-89.75,-179.25,EF
2,-89.75,-178.75,EF
3,-89.75,-178.25,EF
4,-89.75,-177.75,EF
...,...,...,...
92411,83.75,-30.75,ET
92412,83.75,-30.25,ET
92413,83.75,-29.75,ET
92414,83.75,-29.25,ET


In [17]:
place={}
place["Melb"]={}
place["Melb"]["Lat"]=-37.8
place["Melb"]["Lon"]=144.9
place
place_df=pd.DataFrame.from_dict(place, orient='index')
place_df





,Lat,Lon
Melb,-37.8,144.9
